In [1]:
from google.colab import drive
drive.mount('/content/drive')


import os
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW

Mounted at /content/drive


In [2]:
# Paths (adjust if needed)
SEGMENTS_DIR = "/content/drive/MyDrive/files/segments" # where .npy segments are stored
META_CSV = os.path.join(SEGMENTS_DIR, 'metadata.csv')
MODEL_SAVE = '/content/drive/MyDrive/files/best_cnn_rnn.pth'
os.makedirs(SEGMENTS_DIR, exist_ok=True)

In [3]:
# Build metadata if missing
if not os.path.exists(META_CSV):
    records = []
    for f in sorted(glob(os.path.join(SEGMENTS_DIR, '*.npy'))):
        fname = os.path.basename(f)
        # expected pattern: Sxxx_<orig>_segN.npy
        parts = fname.split('_')
        if len(parts) < 2:
            continue
        subj = parts[0]
        session = 1
        # try to detect session in filename
        for p in parts:
            if p.lower().startswith('session'):
                try:
                    session = int(p.replace('session',''))
                except:
                    session = 1
        # subject like S001 -> label 0
        label = int(subj.replace('S','')) - 1
        records.append({'path': f, 'label': label, 'session': session})
    meta = pd.DataFrame.from_records(records)
    meta.to_csv(META_CSV, index=False)
    print('Created metadata.csv with', len(meta), 'entries')
else:
    meta = pd.read_csv(META_CSV)
    print('Loaded metadata.csv with', len(meta), 'entries')

Created metadata.csv with 32029 entries


In [4]:
# Dataset: compute spectrogram per segment on-the-fly
class EEGSpectrogramDataset(Dataset):
    def __init__(self, meta_df, sample_rate=160, n_fft=256, hop=64, freq_range=(1,50), transform=None):
        self.meta = meta_df.reset_index(drop=True)
        self.sample_rate = sample_rate
        self.n_fft = n_fft
        self.hop = hop
        self.freq_range = freq_range
        self.transform = transform

    def __len__(self):
        return len(self.meta)

    def __getitem__(self, idx):
        row = self.meta.iloc[idx]
        arr = np.load(row['path']).astype(np.float32)  # (C, T)
        x = torch.from_numpy(arr)
        # channel-wise STFT -> magnitude
        specs = []
        for ch in range(x.shape[0]):
            stft = torch.stft(x[ch], n_fft=self.n_fft, hop_length=self.hop, win_length=self.n_fft, center=True, return_complex=True)
            mag = stft.abs()
            specs.append(mag.unsqueeze(0))
        spec = torch.cat(specs, dim=0)  # (C, F, Tfr)
        freqs = torch.linspace(0, self.sample_rate / 2, steps=(self.n_fft // 2 + 1))
        fmin, fmax = self.freq_range
        fmask = (freqs >= fmin) & (freqs <= fmax)
        spec = spec[:, fmask, :]
        spec = torch.log1p(spec)
        label = int(row['label'])
        session = int(row['session']) if 'session' in row else 1
        if self.transform:
            spec = self.transform(spec)
        return spec, label, session

In [5]:
# Simple Hybrid Model
# CNN encoder produces timewise features -> GRU processes temporal sequence
class CNNEncoder(nn.Module):
    def __init__(self, in_ch, cnn_channels=(32,64,128)):
        super().__init__()
        layers = []
        prev = in_ch
        for outc in cnn_channels:
            layers += [
                nn.Conv2d(prev, outc, kernel_size=3, padding=1),
                nn.BatchNorm2d(outc),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2,2))
            ]
            prev = outc
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

class HybridCNNRNN(nn.Module):
    def __init__(self, in_ch, n_freq_bins, cnn_channels=(32,64,128), gru_hidden=256, gru_layers=2, n_classes=109, dropout=0.4):
        super().__init__()
        self.encoder = CNNEncoder(in_ch, cnn_channels=cnn_channels)
        # compute feature dim after cnn pooling for freq dimension
        # assume freq dimension is divisible by 2**len(cnn_channels)
        self.freq_reduction = 2 ** len(cnn_channels)
        feat_dim = cnn_channels[-1] * (n_freq_bins // self.freq_reduction)
        self.gru = nn.GRU(input_size=feat_dim, hidden_size=gru_hidden, num_layers=gru_layers, batch_first=True, bidirectional=True)
        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(gru_hidden*2, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, n_classes)
        )

    def forward(self, x):
        # x: (B, C, F, T)
        out = self.encoder(x)  # (B, C', F', T')
        B, C1, Fp, Tp = out.shape
        out = out.permute(0, 3, 1, 2).contiguous()  # (B, Tp, C', F')
        out = out.view(B, Tp, C1 * Fp)  # (B, Tp, feat)
        gru_out, _ = self.gru(out)
        pooled = gru_out.mean(dim=1)
        logits = self.classifier(pooled)
        return logits

In [6]:
# Train / Eval helpers
from sklearn.metrics import accuracy_score, top_k_accuracy_score
import torch.nn.functional as F

def train_epoch(model, loader, optim, device):
    model.train()
    losses = []
    all_preds, all_labels = [], []
    for spec, label, _ in tqdm(loader, desc='train', leave=False):
        spec = spec.to(device)
        label = label.to(device)
        optim.zero_grad()
        out = model(spec)
        loss = F.cross_entropy(out, label)
        loss.backward()
        optim.step()
        losses.append(loss.item())
        preds = out.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds.tolist())
        all_labels.extend(label.cpu().numpy().tolist())
    return np.mean(losses), accuracy_score(all_labels, all_preds)

@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()
    probs_list = []
    labels = []
    for spec, label, _ in tqdm(loader, desc='eval', leave=False):
        spec = spec.to(device)
        out = model(spec)
        probs = F.softmax(out, dim=1).cpu().numpy()
        probs_list.append(probs)
        labels.extend(label.numpy().tolist())
    probs = np.concatenate(probs_list, axis=0)
    preds = probs.argmax(axis=1)
    top1 = accuracy_score(labels, preds)
    try:
        top5 = top_k_accuracy_score(labels, probs, k=5)
    except Exception:
        top5 = None
    return top1, top5


In [7]:
# Prepare splits, dataloaders
from sklearn.model_selection import train_test_split

# Perform a random train-test split on the entire metadata
train_meta, test_meta = train_test_split(meta, test_size=0.2, random_state=42, stratify=meta['label'])

train_meta = train_meta.reset_index(drop=True)
test_meta = test_meta.reset_index(drop=True)

print(f'Train samples: {len(train_meta)}')
print(f'Test samples: {len(test_meta)}')

train_ds = EEGSpectrogramDataset(train_meta)
test_ds = EEGSpectrogramDataset(test_meta)

# quick dimension probe
spec0, _, _ = train_ds[0]
C, n_freq_bins_val, T = spec0.shape # Renamed F to n_freq_bins_val
print('Spec shape per sample:', spec0.shape)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)


Train samples: 25623
Test samples: 6406
Spec shape per sample: torch.Size([64, 79, 6])


/usr/local/lib/python3.12/dist-packages/torch/functional.py:681: UserWarning: A window was not provided. A rectangular window will be applied,which is known to cause spectral leakage. Other windows such as torch.hann_window or torch.hamming_window are recommended to reduce spectral leakage.To suppress this warning and use a rectangular window, explicitly set `window=torch.ones(n_fft, device=<device>)`. (Triggered internally at /pytorch/aten/src/ATen/native/SpectralOps.cpp:836.)
  return _VF.stft(  # type: ignore[attr-defined]


In [8]:
# Instantiate model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HybridCNNRNN(in_ch=C, n_freq_bins=n_freq_bins_val, n_classes=109, cnn_channels=(32,64)).to(device) # Changed F to n_freq_bins_val
optim = AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=8, gamma=0.5)


In [ ]:
# Training loop
best = 0.0
epochs = 10
for ep in range(1, epochs+1):
    loss, train_acc = train_epoch(model, train_loader, optim, device)
    top1, top5 = evaluate(model, test_loader, device)
    scheduler.step()
    print(f'Epoch {ep:02d} loss={loss:.4f} train_acc={train_acc:.3f} test_top1={top1:.3f} test_top5={top5}')
    if top1 > best:
        best = top1
        torch.save(model.state_dict(), MODEL_SAVE)
        print('Saved best model ->', MODEL_SAVE)

print('Training complete. Best test top1:', best)


train:   0%|          | 0/1602 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
# Compute per-subject accuracy and save a simple report
@torch.no_grad()
def per_subject_accuracy(model, loader, device, meta_df):
    model.eval()
    preds = []
    labels = []
    paths = []
    for spec, label, _ in tqdm(loader, desc='per-subject', leave=False):
        spec = spec.to(device)
        out = model(spec)
        p = out.argmax(dim=1).cpu().numpy()
        preds.extend(p.tolist())
        labels.extend(label.numpy().tolist())
    df = pd.DataFrame({'label': labels, 'pred': preds})
    accs = df.groupby('label').apply(lambda d: (d['label']==d['pred']).mean())
    accs = accs.sort_values()
    accs.to_csv(os.path.join(os.path.dirname(MODEL_SAVE), 'per_subject_acc.csv'))
    print('Saved per-subject accuracies.')

In [ ]:
# load best model and run per-subject report
model.load_state_dict(torch.load(MODEL_SAVE, map_location=device))
per_subject_accuracy(model, test_loader, device, test_meta)
print('Report saved near model file.')
